In [2]:
import numpy
import urllib
import scipy.optimize
import random
import math
import copy
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("file:train.json"))
train_set = data[0:180000]
valid_set = data[180000:200000]
print "done"

Reading data...
done


In [3]:
users_items = {}
items_users = {}
#for datum in data:
for datum in train_set:
    u,i = datum['userID'],datum['businessID']
    if not users_items.has_key(u):
        users_items[u] = [(i,datum['rating'])]
    else:
        users_items[u].append((i,datum['rating']))
    if not items_users.has_key(i):
        items_users[i] = [(u,datum['rating'])]
    else:
        items_users[i].append((u,datum['rating']))
sum_rating = 0
count_rating = 0
for u,i in users_items.iteritems():
    for i_info in i:
        sum_rating += i_info[1]
        count_rating += 1
alpha = float(sum_rating)/count_rating

In [4]:
beta_u = {}
for u in users_items.keys():
    count_bias = 0
    count_rate = 0
    for i in users_items[u]:
        count_rate += 1
        count_bias += float(i[1]-alpha)
    beta_u[u]=float(count_bias)/count_rate

In [5]:
beta_i = {}
for i in items_users.keys():
    count_bias = 0
    count_rate = 0
    for u in items_users[i]:
        count_rate += 1
        count_bias += float(u[1]-alpha)
    beta_i[i]=float(count_bias)/count_rate

In [6]:
# create gamma
gamma_u = {}
gamma_i = {}
K = 10
num_users = K
num_items = K
for u in users_items.keys():
    gamma_u[u] = numpy.random.random((1,K))
for i in items_users.keys():
    gamma_i[i] = numpy.random.random((1,K))

In [7]:
for i in users_items.keys():
    for j in range(K):
        gamma_u[i][0][j] = gamma_u[i][0][j] - 0.5
        gamma_u[i][0][j] = gamma_u[i][0][j] * 0.1
for i in items_users.keys():
    for j in range(K):
        gamma_i[i][0][j] = gamma_i[i][0][j] - 0.5
        gamma_i[i][0][j] = gamma_i[i][0][j] * 0.1

In [8]:
def MSE(Alpha,Betauser,Betaitem,gammaU,gammaI):
    square_err = 0
    for u in users_items.keys():
        for i in users_items[u]:
            square_err += (Alpha+Betauser[u]+Betaitem[i[0]]+gammaU[u].dot(gammaI[i[0]].transpose())[0][0]-i[1])**2
    MSE = float(square_err)/len(train_set)
    return MSE
def valid_MSE(Alpha,Betau,Betai,gammau,gammai):
    valid_square_err = 0
    for datum in valid_set:
        u = datum['userID']
        i = datum['businessID']
        try:
            valid_square_err += (Alpha+Betau[u]+Betai[i]+gammau[u].dot(gammai[i].transpose())[0][0]-datum['rating'])**2
        except KeyError:
            valid_square_err += (Alpha-datum['rating'])**2
        #if u in users_items.keys() and i in items_users.keys():
        #    valid_square_err += (Alpha+Betau[u]+Betai[i]-datum['rating'])**2
        #else:
        #    valid_square_err += (Alpha-datum['rating'])**2
    valid_MSE = float(valid_square_err)/len(valid_set)
    return valid_MSE

In [23]:
def update_alpha_beta(Alpha,Betauser,Betaitem,gammaU,gammaI,lam):
    #total = len(train_set)
    # Update alpha
    sum_for_alpha = 0
    for u in users_items.keys():
        for i in users_items[u]:
            sum_for_alpha += i[1] - Betauser[u] - Betaitem[i[0]] - gammaU[u].dot(gammaI[i[0]].transpose())[0][0]
    Alpha = float(sum_for_alpha) / len(train_set)
    # Update beta_user
    for u in users_items.keys():
        sum_for_betauser = 0
        count_item = 0
        for i in users_items[u]:
            count_item += 1
            sum_for_betauser += i[1] - Alpha - Betaitem[i[0]] - gammaU[u].dot(gammaI[i[0]].transpose())[0][0]
        Betauser[u] = float(sum_for_betauser) / (lam + count_item)
    # Update beta_item
    for i in items_users.keys():
        sum_for_betaitem = 0
        count_user = 0
        for u in items_users[i]:
            count_user += 1
            sum_for_betaitem += u[1] - Alpha - Betauser[u[0]] - gammaU[u[0]].dot(gammaI[i].transpose())[0][0]
        Betaitem[i] = float(sum_for_betaitem) / (lam + count_user)
    return Alpha,Betauser,Betaitem

def update_gamma_u(Alpha,Betauser,Betaitem,gammaU,gammaI,K,lam,rate,index):
    total = len(train_set)
    # Update gammaU
    for u in users_items.keys():
        sum_for_gammaU = 0
        update_flag = bool(random.getrandbits(1))
        if (update_flag):
            for i in users_items[u]:
                sum_for_gammaU += i[1] - Alpha - Betauser[u] - Betaitem[i[0]] - gammaU[u].dot(gammaI[i[0]].transpose())[0][0]
                for count_gamma in [index]:
                    diff = float(-2 * sum_for_gammaU * gammaI[i[0]][0][count_gamma]) / total
                    gammaU[u][0][count_gamma] = gammaU[u][0][count_gamma] - rate * diff
    return gammaU

def update_gamma_i(Alpha,Betauser,Betaitem,gammaU,gammaI,K,lam,rate,index):
    total = len(train_set)
    # Update gammaI
    for i in items_users.keys():
        sum_for_gammaI = 0
        update_flag = bool(random.getrandbits(1))
        if (update_flag):
            for u in items_users[i]:
                sum_for_gammaI += u[1] - Alpha - Betauser[u[0]] - Betaitem[i] - gammaU[u[0]].dot(gammaI[i].transpose())[0][0]
                for count_gamma in [index]:
                    diff = float(-2 * sum_for_gammaI * gammaU[u[0]][0][count_gamma]) / total
                    gammaI[i][0][count_gamma] = gammaI[i][0][count_gamma] - rate * diff
    return gammaI

In [9]:
for lam in [4]:
    a = alpha
    betau = beta_u.copy()
    betai = beta_i.copy()
    gammau = copy.deepcopy(gamma_u)
    gammai = copy.deepcopy(gamma_i)
    print 'Initial MSE = ' + str(MSE(a,betau,betai,gammau,gammai))
    print 'lam\tIter\tMSE'
    count = 0
    flag = 1
    mse_tmp = 999
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_pre = mse_tmp
        mse_tmp = MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_pre
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammau_old = copy.deepcopy(gammau)
        for count_index in range(K):
            while(flag):
                count += 1
                gammau = update_gamma_u(a,betau,betai,gammau,gammai,K,lam,10,count_index)
                mse_pre = mse_tmp
                mse_tmp = MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammau = copy.deepcopy(gammau_old)
                    mse_tmp = mse_pre
                    break
                if (mse_pre - mse_tmp < 0):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammai_old = copy.deepcopy(gammai)
        for count_index in range(K):
            while(flag):
                count += 1
                gammai = update_gamma_i(a,betau,betai,gammau,gammai,K,lam,10,count_index)
                mse_old = mse_pre
                mse_pre = mse_tmp
                mse_tmp = MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammai = copy.deepcopy(gammai_old)
                    mse_tmp = mse_old
                    break
                if (mse_pre - mse_tmp < 0.0000001):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammau_old = copy.deepcopy(gammau)
        for count_index in range(K):
            while(flag):
                count += 1
                gammau = update_gamma_u(a,betau,betai,gammau,gammai,K,lam,1,count_index)
                mse_pre = mse_tmp
                mse_tmp = MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammau = copy.deepcopy(gammau_old)
                    mse_tmp = mse_pre
                    break
                if (mse_pre - mse_tmp < 0):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammai_old = copy.deepcopy(gammai)
        for count_index in range(K):
            while(flag):
                count += 1
                gammai = update_gamma_i(a,betau,betai,gammau,gammai,K,lam,1,count_index)
                mse_old = mse_pre
                mse_pre = mse_tmp
                mse_tmp = MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammai = copy.deepcopy(gammai_old)
                    mse_tmp = mse_old
                    break
                if (mse_pre - mse_tmp < 0.0000001):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.0000001):
            print 'converge'
            break

Initial MSE = 0.478338456284
lam	Iter	MSE
lam=4	1	0.444001277338
lam=4	2	0.435830542346
lam=4	3	0.434483330082
lam=4	4	0.433985301168
lam=4	5	0.433751617033
lam=4	6	0.433628711816
lam=4	7	0.433558979011
lam=4	8	0.43351732684
lam=4	9	0.433491710316
lam=4	10	0.433475812195
lam=4	11	0.433466032798
lam=4	12	0.433460176284
lam=4	13	0.433456839297
lam=4	14	0.433455102146
lam=4	15	0.433454357283
lam=4	16	0.433454205121
lam=4	17	0.433454386379
not descend
lam=4	18	0.433454159753
lam=4	19	0.433454114392
lam=4	20	0.433454069038
lam=4	21	0.433454023693
lam=4	22	0.433453978354
lam=4	23	0.433453933024
lam=4	24	0.433453887701
lam=4	25	0.433453842386
lam=4	26	0.433453797078
lam=4	27	0.433453751778
lam=4	28	0.433453706485
lam=4	29	0.433453661201
lam=4	30	0.433453615923
lam=4	31	0.433453570654
lam=4	32	0.433453525392
lam=4	33	0.433453480137
lam=4	34	0.43345343489
lam=4	35	0.433453389651
lam=4	36	0.43345334442
lam=4	37	0.433453299196
lam=4	38	0.433453253979
lam=4	39	0.43345320877
lam=4	40	0.433453163569

KeyboardInterrupt: 

In [ ]:
### Gradient descent ###
# Objective
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]
# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])

scipy.optimize.fmin_l_bfgs_b(f, [gamma_u], fprime, args = (X, y, 0))

In [39]:
for lam in [4]:
    #a = alpha
    #betau = beta_u.copy()
    #betai = beta_i.copy()
    #gammau = copy.deepcopy(gamma_u)
    #gammai = copy.deepcopy(gamma_i)
    print 'Initial MSE = ' + str(valid_MSE(a,betau,betai,gammau,gammai))

Initial MSE = 0.591809253517


In [24]:
for lam in [4]:
    #a = alpha
    #betau = beta_u.copy()
    #betai = beta_i.copy()
    #gammau = copy.deepcopy(gamma_u)
    #gammai = copy.deepcopy(gamma_i)
    print 'Initial MSE = ' + str(valid_MSE(a,betau,betai,gammau,gammai))
    print 'lam\tIter\tvalid_MSE'
    count = 0
    flag = 1
    mse_tmp = 999
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_pre = mse_tmp
        mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_pre
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammau_old = copy.deepcopy(gammau)
        for count_index in range(K):
            while(flag):
                count += 1
                gammau = update_gamma_u(a,betau,betai,gammau,gammai,K,lam,1000,count_index)
                mse_pre = mse_tmp
                mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammau = copy.deepcopy(gammau_old)
                    mse_tmp = mse_pre
                    break
                if (mse_pre - mse_tmp < 0):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammai_old = copy.deepcopy(gammai)
        for count_index in range(K):
            while(flag):
                count += 1
                gammai = update_gamma_i(a,betau,betai,gammau,gammai,K,lam,10,count_index)
                mse_old = mse_pre
                mse_pre = mse_tmp
                mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammai = copy.deepcopy(gammai_old)
                    mse_tmp = mse_old
                    break
                if (mse_pre - mse_tmp < 0.0000001):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammau_old = copy.deepcopy(gammau)
        for count_index in range(K):
            while(flag):
                count += 1
                gammau = update_gamma_u(a,betau,betai,gammau,gammai,K,lam,10,count_index)
                mse_pre = mse_tmp
                mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammau = copy.deepcopy(gammau_old)
                    mse_tmp = mse_pre
                    break
                if (mse_pre - mse_tmp < 0):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.00000001):
            print 'converge'
            break
    while(flag):
        gammai_old = copy.deepcopy(gammai)
        for count_index in range(K):
            while(flag):
                count += 1
                gammai = update_gamma_i(a,betau,betai,gammau,gammai,K,lam,10,count_index)
                mse_old = mse_pre
                mse_pre = mse_tmp
                mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
                print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
                if (mse_pre < mse_tmp):
                    print 'not descend'
                    gammai = copy.deepcopy(gammai_old)
                    mse_tmp = mse_old
                    break
                if (mse_pre - mse_tmp < 0.0000001):
                    print 'converge'
                    break
        break
    while(flag):
        count += 1
        a_old = a
        betau_old = betau.copy()
        betai_old = betai.copy()
        a,betau,betai = update_alpha_beta(a,betau,betai,gammau,gammai,lam)
        mse_old = mse_pre
        mse_pre = mse_tmp
        mse_tmp = valid_MSE(a,betau,betai,gammau,gammai)
        print 'lam='+str(lam)+'\t'+str(count) + '\t' + str(mse_tmp)
        if (mse_pre < mse_tmp):
            print 'not descend'
            a = a_old
            betau = betau_old.copy()
            betai = betai_old.copy()
            mse_tmp = mse_old
            break
        if (mse_pre - mse_tmp < 0.0000001):
            print 'converge'
            break

Initial MSE = 0.465332701332
lam	Iter	valid_MSE
lam=4	1	0.591809253517
lam=4	2	0.597921723749
not descend
lam=4	3	0.591809974565
not descend


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
predictions = open("predictions_Rating_1118.txt", 'w+')
for l in open("pairs_Rating.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  try:
    predictions.write(u + '-' + i + ',' + str(a+betau[u]+betai[i]+gammau[u].dot(gammai[i].transpose())[0][0]) + '\n')
  except KeyError:
    predictions.write(u + '-' + i + ',' + str(a) + '\n')